# Find Products purchased by customers who bought Product A

Related article: https://www.dataenlightened.com/posts/products-purchased-by-customer-who-purchased-product-a/


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# BigQuery SDK Library setup
%load_ext google.cloud.bigquery

import os
from google.cloud import bigquery
from google.oauth2 import service_account


## Load Google Analytics 360 data from BigQuery

In [2]:
def load_data_from_bq(sql, credential_json_file):

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credential_json_file
    credentials = service_account.Credentials.from_service_account_file(credential_json_file)

    bqclient = bigquery.Client(credentials=credentials)

    # Download query results.
    query_string = sql

    df = (
        bqclient.query(query_string)
            .result()
            .to_dataframe(
            # Optionally, explicitly request to use the BigQuery Storage API. As of
            # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
            # API is used by default.
            create_bqstorage_client=True,
        )
    )

    return df

In [4]:
my_ga_dataset = "bigquery-public-data.google_analytics_sample"  # Replace with your own GA 360 BigQuery dataset

credentials = "credentials.json" # Replace this with your own Bigquery Service Credentials JSON

# SKU = GGOEGDWR015799
# Product Name = 15 Oz Ceramic Mug

sql = f"""
SELECT
  fullVisitorID,
  productSKU AS Other_Purchased_SKU,
  Other_Product_Name,
  COUNT(productSKU) AS quantity
FROM (
  SELECT
    fullVisitorId,
    product.productSKU,
    product.v2ProductName AS Other_Product_Name,
    hits.eCommerceAction.action_type
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits,
    UNNEST(product) AS product
  WHERE
    fullVisitorId IN (
    SELECT
      fullVisitorId
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
      UNNEST(hits) AS hits,
      UNNEST(product) AS product
    WHERE
      product.productSKU = 'GGOEGDWR015799'   # SKU for 15 oz Ceramic Mug
      AND hits.eCommerceAction.action_type = '6'
    GROUP BY
      fullVisitorId )
    AND product.productSKU IS NOT NULL
    AND product.productSKU != 'GGOEGDWR015799'  # SKU for 15 oz Ceramic Mug
    AND hits.eCommerceAction.action_type = '6' )
GROUP BY
  fullVisitorID,
  Other_Purchased_SKU,
  Other_Product_Name
ORDER BY
  quantity DESC;
"""

df = load_data_from_bq(sql=sql, credential_json_file=credentials)

In [5]:
df.head()

,fullVisitorID,Other_Purchased_SKU,Other_Product_Name,quantity
0,1956307607572137989,GGOEGDHR018499,Google 22 oz Water Bottle,14
1,4988517937139937145,GGOEGDHC074099,Google 17oz Stainless Steel Sport Bottle,14
2,1956307607572137989,GGOEGDHC018299,Google 22 oz Water Bottle,14
3,9377429831454005466,GGOEGGOA017399,Maze Pen,12
4,043584487743669327,GGOEGGCX056299,Gift Card - $25.00,12


## What are the 5 most bought products among all these products?

In [29]:
# Sort products by quantity purchased most to least
df_grouped = df.groupby(['Other_Product_Name'])['Other_Product_Name', 'quantity'].sum().reset_index().copy()

# Get the Top 5 bought products
df_grouped.sort_values('quantity', ascending=False).nlargest(5, 'quantity')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Other_Product_Name,quantity
200,Google Sunglasses,230
77,Engraved Ceramic Google Mug,154
86,Google 22 oz Water Bottle,132
129,Google Laptop and Cell Phone Stickers,130
284,Recycled Paper Journal Set,84


## What are the 5 least bought products among all these products?

In [30]:
# Get the Least 5 bought products
df_grouped.sort_values('quantity', ascending=True).nsmallest(5, 'quantity')

,Other_Product_Name,quantity
103,Google Bongo Cupholder Bluetooth Speaker,2
274,Metal Texture Roller Pen,2
233,Google Women's Short Sleeve Hero Tee Heather,2
229,Google Women's Short Sleeve Badge Tee Grey,2
75,Eco-Aware Recycled Journal Combo,2


## Fast-moving Products: Customers promptly buy them without doing much research on them

In [ ]:
df.sort_values('avg_hits', ascending=True).head(10)

,ProductSku,ProductName,avg_hits
371,GGOEGAAX0127,Android 24 oz Button Lid Sport Water Bottle,0.027006
1319,GGOEGDWR015799,15 oz Ceramic Mug,0.031949
802,GGOEGBMC056599,Google Small Waterproof Duffel,0.057475
1783,GGOEGESQ016799,Google Magnetic Mini Flood Flashlight,0.060445
1997,GGOEGAAX0327,Google Men's Long & Lean Tee Grey,0.070234
1992,GGOEGOXQ016399,Google Badge Pull,0.077317
510,GGOEGAAX0098,Google 7-inch Dog Flying Disc,0.079084
722,9182575,Android Men's Zip Hoodie,0.079710
1018,GGOEGAAX0328,Google Men's Long & Lean Tee Charcoal,0.082099
1304,GGOEGHGH019699,Google Sunglasses Green,0.083260


# Find Products purchased by customers who purchased product A

In [1]:
sql = """SELECT
  fullVisitorID,
  productSKU AS Other_Purchased_SKU,
  Other_Product_Name,
  COUNT(productSKU) AS quantity
FROM (
  SELECT
    fullVisitorId,
    product.productSKU,
    product.v2ProductName AS Other_Product_Name,
    hits.eCommerceAction.action_type
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits,
    UNNEST(product) AS product
  WHERE
    fullVisitorId IN (
    SELECT
      fullVisitorId
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
      UNNEST(hits) AS hits,
      UNNEST(product) AS product
    WHERE
      product.productSKU = 'GGOEGDWR015799'   # SKU for 15 oz Ceramic Mug
      AND hits.eCommerceAction.action_type = '6'
    GROUP BY
      fullVisitorId )
    AND product.productSKU IS NOT NULL
    AND product.productSKU != 'GGOEGDWR015799'  # SKU for 15 oz Ceramic Mug
    AND hits.eCommerceAction.action_type = '6' )
GROUP BY
  fullVisitorID,
  Other_Purchased_SKU,
  Other_Product_Name
ORDER BY
  quantity DESC;
"""


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0bf64fc0-3489-421f-847e-b901bd3270a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>